# Multi-Agent Architecture with Semantic Kernel and Azure API Management

## Workshop for RWS: Building a Proof of Concept

This workshop guides you through building a multi-agent architecture that leverages Semantic Kernel for orchestrating agents and Azure API Management (APIM) as an AI Gateway for function calling against backend services like Azure Functions and databases.

### Architecture Overview

```
┌─────────────────┐     ┌───────────────┐     ┌─────────────────┐     ┌────────────────┐
│                 │     │               │     │                 │     │                │
│  Semantic Kernel│     │  Azure API    │     │  Integration    │     │  Data Sources  │
│  Multi-Agents   │────▶│  Management   │────▶│  Services       │────▶│              │
│                 │     │  (AI Gateway) │     │  (Functions)    │     │                │
└─────────────────┘     └───────────────┘     └─────────────────┘     └────────────────┘
```

### What You'll Learn

1. How to set up Semantic Kernel with multiple specialized agents
2. How to connect agents to Azure API Management for function calling
3. How to orchestrate agent collaboration using different strategies
5. How to build specialized agents performing different tasks (RAG with AI Search, function calling, etc.)

### Implementation Approach

The implementation code is organized into Python modules in the `rws-app` directory for better maintainability and reuse. This notebook will focus on explaining concepts and demonstrating how to use these modules.

Let's begin.

## Step 1: Environment Setup and Prerequisites

First, let's install the necessary packages and set up our environment. We'll need Semantic Kernel and other supporting libraries.

In [ ]:
# Install required packages
! pip install semantic-kernel==1.29.0 azure-identity python-dotenv requests pyodbc

We'll use Python modules from the `rws-app` directory to organize our code. Let's import these modules:

In [ ]:
import os
import sys

# Add the following directories to the Python path
sys.path.append(os.path.abspath("./rws-app"))
sys.path.append(os.path.abspath("../../../shared"))

# Import our utility functions
from utils import check_and_load_environment, display_environment_variables

required_vars = [
    "AZURE_OPENAI_ENDPOINT",
    "AZURE_OPENAI_API_KEY",
    "AZURE_OPENAI_MODEL_DEPLOYMENT_NAME",
    "APIM_GATEWAY_URL",
    "APIM_SUBSCRIPTION_KEY",
]
# Make sure we have the necessary environment variables
check_and_load_environment(required_vars)

# Display the environment variables (masking sensitive ones)
display_environment_variables()

To use this workshop, you need to create a `.env` file with the following variables:

```
AZURE_OPENAI_ENDPOINT='[YOUR_ENDPOINT]'
AZURE_OPENAI_API_KEY='[YOUR_API_KEY]'
AZURE_OPENAI_MODEL_DEPLOYMENT_NAME='gpt-4o'
APIM_GATEWAY_URL='[YOUR_APIM_GATEWAY_URL]'
APIM_SUBSCRIPTION_KEY='[YOUR_SUBSCRIPTION_KEY]'
```

These will connect to your Azure resources, including Azure OpenAI and API Management.

## Step 2: Setting Up Semantic Kernel

Now, let's set up a Semantic Kernel instance that will be the foundation for our agent system. We'll use our `kernel_setup` module to create the kernel with the appropriate AI service.

In [ ]:
# Import the kernel setup function
from kernel_setup import create_kernel_with_service

# Create a main kernel for our agents
kernel = create_kernel_with_service(service_id="chat-completion")
print("Kernel created successfully!")

## Step 3: Connecting to Azure API Management

Now, let's set up functions to call the API endpoints through APIM. These will be registered with our kernel and made available to the agents. 

The `ApiManagementPlugin` class in the `api_plugin.py` module provides:
- A connection to the API Management gateway
- Functions for getting weather data
- Functions for querying SQL databases
- Functions for retrieving sales data by region

In [ ]:
# Import our API Management Plugin
from api_plugin import ApiManagementPlugin

# Create an instance of the plugin
apim_plugin = ApiManagementPlugin()

# Add to kernel
kernel.add_plugin(apim_plugin, plugin_name="ApiManagement")
functions = kernel.get_plugin("ApiManagement").functions
print("API Management plugin registered with functions:")
print([f.name for f in functions.values()])

Let's examine how the API plugin works. It leverages Semantic Kernel's function calling capabilities to allow LLMs to interact with external systems through API Management. This approach provides several benefits:

1. **Abstraction**: The agents don't need to know the details of how APIs are implemented
2. **Security**: API keys are managed securely through API Management
3. **Centralization**: All API calls go through a common gateway
4. **Monitoring**: API calls can be monitored and analyzed

Each function in the plugin is decorated with `@kernel_function` and includes parameter annotations to help the LLM understand how to use them.

## Step 4: Creating Specialized Agents

Now, let's create specialized agents that can perform different tasks using the functions we've registered. Each agent will have a specific role and expertise.

We have defined three agents:
1. **DataAnalyst**: Analyzes sales data and provides insights
2. **EnvironmentalExpert**: Provides weather information and its impact on agriculture
3. **BusinessAdvisor**: Synthesizes information and provides strategic recommendations

In [ ]:
# Import our agent creation functions
from agents import (
    create_infrastructure_analyst_agent,
    create_water_management_expert_agent,
    create_strategic_advisor_agent,
    create_knowledge_agent,
    create_research_synthesis_agent,
)

from semantic_kernel.functions import KernelArguments
from semantic_kernel.connectors.ai import FunctionChoiceBehavior

# Create settings with auto function calling enabled
settings = kernel.get_prompt_execution_settings_from_service_id(
    service_id="chat-completion"
)

settings.function_choice_behavior = FunctionChoiceBehavior.Auto()

# Create our specialized agents
infrastructure_analysis_agent = create_infrastructure_analyst_agent(kernel, settings)
water_management_expert_agent = create_water_management_expert_agent(kernel, settings)
strategic_advisor_agent = create_strategic_advisor_agent(kernel, settings)
knowledge_agent = create_knowledge_agent(kernel, settings)
research_synthesis_agent = create_research_synthesis_agent(kernel, settings)

# Store all agents in a list for convenience
agents = [
    infrastructure_analysis_agent,
    water_management_expert_agent,
    strategic_advisor_agent,
    knowledge_agent,
    research_synthesis_agent,
]

print(
    f"Created specialized agents: {', '.join([agent.name for agent in agents])}"
)

Each agent is created with specific instructions that guide its behavior:

1. **DataAnalyst** focuses on retrieving and analyzing data from the SQL database
2. **EnvironmentalExpert** specializes in weather data and its implications
3. **BusinessAdvisor** synthesizes information and provides strategic recommendations

By creating specialized agents, we can benefit from:
- **Division of labor**: Each agent focuses on what it does best
- **Expertise**: Specialized prompts create more expert behavior
- **Clearer responsibilities**: Each agent has a defined role

## Step 5: Testing Individual Agents

Before creating a multi-agent system, let's test each agent individually to make sure they can perform their specialized tasks.
Here is a list of possible questions, designed to leverage on agents' expertise and access to external data.

Here are meaningful questions for each agent that leverage the SQL function endpoints:

For **Infrastructure Analysis Agent**:
1. "Can you analyze the critical infrastructure assets in Noord-Nederland and recommend prioritization for maintenance based on safety ratings and inspection findings?"
2. "What patterns do you see in safety inspection reports for bridges across different regions, and what preventive measures would you recommend?"
3. "Based on the active maintenance projects and safety ratings, which infrastructure types need the most immediate attention?"

For **Water Management Expert Agent**:
1. "Looking at safety inspection data for water-related infrastructure in coastal regions, what trends do you notice about maintenance needs?"
2. "How do safety ratings of water management assets compare across different regions, and what might explain these differences?"
3. "Based on recent inspection findings, what improvements would you recommend for water infrastructure maintenance protocols?"

For **Strategic Advisor Agent**:
1. "Using the asset statistics and safety inspection data, what strategic recommendations would you make for long-term infrastructure investment?"
2. "How should we adjust our maintenance strategy based on the distribution of critical assets across regions?"
3. "What policy recommendations would you make based on the correlation between inspection frequency and safety ratings?"

For **Knowledge Agent**:
1. "Can you analyze the safety inspection findings across different asset types and create a knowledge base of common issues and solutions?"
2. "What insights can you draw from comparing maintenance projects' priorities with their actual safety ratings?"
3. "How do our infrastructure maintenance practices compare to industry best practices, based on our inspection data?"

For **Research Synthesis Agent**:
1. "Can you synthesize the safety inspection findings and maintenance project data to identify emerging infrastructure challenges?"
2. "What research implications can you draw from the relationship between asset age and safety ratings across different infrastructure types?"
3. "Based on our inspection and maintenance data, what research areas should we prioritize for improving infrastructure resilience?"

In [ ]:
# Import the test_agent function
from collaboration import test_agent

# Test the Data Analyst agent
await test_agent(infrastructure_analysis_agent, "Can you analyze the critical infrastructure assets in Noord-Nederland and recommend prioritization for maintenance based on safety ratings and inspection findings?")

In [ ]:
# Test the Environmental Expert agent
await test_agent(water_management_expert_agent, "How do safety ratings of water management assets compare across different regions, and what might explain these differences?")

## Step 6: Setting Up Multi-Agent Collaboration - Sequential Approach

Now that we've tested each agent individually, let's create a multi-agent system where agents can collaborate to solve complex problems. We'll start with a simple sequential (round-robin) approach where agents take turns in a fixed order.

In [ ]:
# Import the collaboration functions
from collaboration import create_sequential_group, run_group_chat

# First, create a sequential (round-robin) collaboration
sequential_group = create_sequential_group(agents, max_iterations=6)  # 2 rounds of all 3 agents

print(f"Created a sequential group chat with {len(sequential_group.agents)} agents")
print(f"Maximum iterations: {sequential_group.termination_strategy.maximum_iterations}")

In [ ]:
# Test the sequential group with a complex question
complex_query = "Given the current sales data and weather conditions in Europe, what strategic recommendations can you provide for our agricultural product sales in the next quarter?"

chat_history = await run_group_chat(sequential_group, complex_query)

### Understanding the Sequential Approach

In the sequential approach:

1. Agents take turns in the order they were added to the group
2. Each agent sees the full conversation history when generating its response
3. The conversation continues until it reaches the maximum number of iterations

This approach is simple and ensures each agent gets an equal opportunity to contribute. However, it may not be the most efficient for all tasks, as some tasks might benefit from a more structured workflow.

## Step 7: Creating a Custom Workflow for Specialized Collaboration

Now, let's create a more tailored collaboration pattern using a custom workflow. This allows us to define exactly which agent speaks in what order, creating a process that matches our business logic.

In [ ]:
# Import the fixed workflow creation function
from collaboration import create_fixed_workflow_chat

# Define our specific workflow for agricultural decision-making:
# First gather data, then analyze environmental conditions, and finally provide business recommendations
agricultural_workflow = [
    "ResearchSynthesisAgent", # First get data
    "WaterManagementExpert",  # Then check environmental conditions
    "StrategicAdvisor",       # Provide initial recommendations
    "ResearchSynthesisAgent", # Do deeper data analysis based on recommendations
    "StrategicAdvisor"        # Final strategic recommendations
]

# Create the workflow chat
workflow_chat = create_fixed_workflow_chat(
    agents=[research_synthesis_agent, water_management_expert_agent, strategic_advisor_agent],
    workflow_sequence=agricultural_workflow,
    max_iterations=len(agricultural_workflow),
)

print(f"Created fixed workflow chat with sequence: {' → '.join(agricultural_workflow)}")


# # Import the fixed workflow creation function
# from collaboration import create_fixed_workflow_chat

# # Define our specific workflow for agricultural decision-making:
# # First gather data, then analyze environmental conditions, and finally provide business recommendations
# agricultural_workflow = [
#     "DataAnalyst",          # First get sales data
#     "EnvironmentalExpert",  # Then check environmental conditions
#     "BusinessAdvisor",      # Provide initial recommendations
#     "DataAnalyst",          # Do deeper data analysis based on recommendations
#     "BusinessAdvisor"       # Final strategic recommendations
# ]

# # Create the workflow chat
# workflow_chat = create_fixed_workflow_chat(
#     agents=[researchSynthesisAgent, environmental_agent, advisor_agent],
#     workflow_sequence=agricultural_workflow,
#     max_iterations=len(agricultural_workflow),
# )

# print(f"Created fixed workflow chat with sequence: {' → '.join(agricultural_workflow)}")

In [ ]:
# Test our custom workflow with a strategic business question
strategic_query = "How should we adapt our planting and distribution strategies for tomato seeds in Europe next season given current sales data and environmental trends?"

chat_history = await run_group_chat(workflow_chat, strategic_query)

### Understanding the Custom Workflow Approach

In the custom workflow approach:

1. We define a specific sequence of agents that mimics a business process
2. The workflow follows steps: data gathering → environmental analysis → initial recommendations → deeper analysis → final recommendations
3. Each agent still sees the full conversation history

This approach provides more control over the collaboration process and can lead to more predictable and structured outputs. It's particularly useful when you have a well-defined process that you want to follow.

## Step 8: Using Azure AI Agents

So far, we've used Semantic Kernel's `ChatCompletionAgent` for our multi-agent system. Now, let's explore using the Azure AI Agent service, which offers additional capabilities like persistent assistants and more advanced tool integration.

Azure AI Agents can be particularly useful when you need:
- Persistent agents that maintain state across sessions
- More complex tool calling capabilities
- Integration with Azure's broader AI ecosystem

Let's start by importing our Azure AI Agent functions:

In [ ]:
# Import the Azure AI Agent functions
from azure_ai_agent import (
    create_data_analyst_azure_agent,
    create_environmental_expert_azure_agent,
    create_business_advisor_azure_agent
)

Now, let's define the function specifications that will be used by our Azure AI Agents. In a real implementation, you would register your functions with the Azure AI Agent service, but for demonstration purposes, we'll define them in the notebook:

In [ ]:
# Define tools that will be available to the Azure AI Agents
available_tools = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "Get weather information for a location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The location to get weather for (city name)"
                    },
                    "unit": {
                        "type": "string",
                        "description": "Temperature unit: 'celsius' or 'fahrenheit'",
                        "default": "celsius"
                    }
                },
                "required": ["location"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "execute_sql_query",
            "description": "Execute a SQL query against the database",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "The SQL query to execute"
                    }
                },
                "required": ["query"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_sales_by_region",
            "description": "Get sales data for a specific region",
            "parameters": {
                "type": "object",
                "properties": {
                    "region_name": {
                        "type": "string",
                        "description": "Optional name of the region to filter by"
                    }
                }
            }
        }
    }
]

Now, let's create our specialized Azure AI Agents:

In [ ]:
try:
    # Create specialized Azure AI Agents
    print("Creating specialized Azure AI Agents...")
    data_analyst_azure = create_data_analyst_azure_agent(tools=available_tools)
    environmental_expert_azure = create_environmental_expert_azure_agent(tools=[available_tools[0]])  # Only weather tool
    business_advisor_azure = create_business_advisor_azure_agent()  # No tools, just synthesizes information
    
    # Store all agents in a list for convenience
    azure_agents = [data_analyst_azure, environmental_expert_azure, business_advisor_azure]
    
    print("Azure AI Agents created successfully!")
except Exception as e:
    print(f"Error creating Azure AI Agents: {str(e)}")
    print("This part of the workshop requires setting up Azure AI Agent resources. Check the Azure AI Services docs for more details.")

### Testing Azure AI Agents

If you've successfully created the Azure AI Agents, you can test them individually. If not, please refer to the Azure AI Agent setup documentation and review the `run_azure_agents.py` script for a complete demonstration.

In [ ]:
# Define a test function for Azure AI Agents
async def test_azure_agent(agent, prompt):
    """Test an individual Azure AI Agent with a prompt.
    
    Args:
        agent: The Azure AI Agent to test
        prompt: The prompt to send to the agent
    """
    print(f"\n=== Testing Azure Agent: {agent.assistant_params.get('display_name', 'Unknown')} ===\n")
    print(f"User: {prompt}\n")
    
    # Get response from the agent
    response = await agent.invoke(prompt=prompt)
    
    print(f"Agent: {response}\n")
    print("=== Test Complete ===\n")
    
    return response

# Try to test the Azure AI Agents
try:
    await test_azure_agent(data_analyst_azure, "Show me the total sales for each region.")
except NameError:
    print("Azure AI Agents not available. Skipping test.")
except Exception as e:
    print(f"Error testing Azure AI Agent: {str(e)}")

### Comparing Semantic Kernel Agents and Azure AI Agents

Both Semantic Kernel Agents and Azure AI Agents offer powerful capabilities for building agent systems, but they have different strengths:

**Semantic Kernel Agents**:
- **Flexibility**: More control over the agent architecture and behavior
- **Local state management**: Easier to integrate with your application state
- **MultiAgent orchestration**: Built-in support for agent collaboration
- **Open-source**: Can be customized and extended as needed

**Azure AI Agents**:
- **Persistence**: Agents and their state persist across sessions
- **Managed service**: Less code to maintain, more robust
- **Advanced capabilities**: Built-in features like file handling, code execution
- **Scalability**: Handles concurrent users more easily

### Combining Both Approaches

For many applications, a hybrid approach works best:

1. Use **Azure AI Agents** for persistent assistants that need rich tool integration
2. Use **Semantic Kernel Agents** for orchestrating collaboration between multiple agents
3. Combine them by using Semantic Kernel's integration with Azure AI Agents

## Step 8: Building Advanced Multi-Agent Systems for Real-World Applications

The techniques demonstrated in this workshop can be extended to build sophisticated multi-agent systems for various enterprise scenarios. Here are some ways to develop this further:

### Enhanced Analytics for Agricultural Data

1. **Time-Series Analysis**: Build agents that can analyze seasonal patterns in agricultural data
2. **Predictive Models**: Create specialized agents for crop yield prediction based on historical data
3. **Market Analysis**: Integrate with commodity market data for pricing strategy recommendations
4. **Risk Assessment**: Develop agents focused on identifying risks (weather, disease, market fluctuations)

### Deployment Options

The system can be deployed in various ways:

1. **Web Application**: Expose the multi-agent system through a web interface
2. **API Service**: Provide the multi-agent capabilities as an API service
3. **Integration with Existing Systems**: Connect with ERP, CRM, or other enterprise systems
4. **Mobile Applications**: Extend access to field workers through mobile apps

### Next Steps

To build a production-ready system, consider:

1. **Agent Persistence**: Implement a database to store agent states and conversation history
2. **Authentication & Authorization**: Add security layers for different users and roles
3. **Logging & Monitoring**: Implement comprehensive logging for all agent actions
4. **Error Handling**: Create robust error handling and recovery mechanisms
5. **Testing & Validation**: Develop thorough testing protocols for your multi-agent system
6. **Scalability**: Design the system to handle increased load and more complex queries

## Conclusion

In this workshop, we've built a complete multi-agent system that leverages Semantic Kernel, Azure AI Agents, and Azure API Management to create a powerful, flexible architecture.

We've covered:

1. Setting up a Semantic Kernel with Azure OpenAI
2. Creating API Management plugins for function calling
3. Designing specialized agents with different roles
4. Implementing different collaboration strategies between agents
5. Building custom workflows for domain-specific processes
6. Integrating Azure AI Agents for enhanced capabilities

The code is organized into maintainable modules in the `rws-app` directory, making it easy to extend and customize for your specific needs.

For further exploration, you can:
- Add more agents with different specializations
- Implement more complex workflow patterns
- Connect to additional APIs through API Management
- Expand the system with more sophisticated reasoning capabilities

The combination of Semantic Kernel's agent architecture, Azure AI Agents, and Azure API Management's function calling capabilities provides a powerful foundation for building intelligent, multi-agent systems that can solve complex real-world problems.